In [1]:
from flask import Flask, session, request
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

In [2]:
app = Flask(__name__)
app.secret_key = 'your_secret_key'

In [3]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Sanvik@5842",
    database="tryagain"
)

In [4]:
cursor = connection.cursor()


In [5]:
df=pd.read_csv("diabetes_prediction_dataset.csv")

In [6]:
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [7]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [8]:
columns_drop = ['gender', 'smoking_history']
df.drop(columns=columns_drop, inplace=True)

In [9]:
X = df.drop('diabetes', axis=1)
y = df['diabetes']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
from sklearn.feature_selection import SelectKBest, chi2
k_best = 5  
selector = SelectKBest(chi2, k=k_best)
X_train_selected = selector.fit_transform(X_train, y_train)
selected_indices = selector.get_support(indices=True)
selected_features = X_train.columns[selected_indices]

In [12]:
print(selected_features)

Index(['age', 'hypertension', 'bmi', 'HbA1c_level', 'blood_glucose_level'], dtype='object')


In [13]:
from sklearn.tree import DecisionTreeClassifier
dtr=DecisionTreeClassifier(random_state=42)
dtr.fit(X_train_selected,y_train)

DecisionTreeClassifier(random_state=42)

In [14]:
X_test_selected = selector.transform(X_test)
first_20_rows = X_test_selected[:20, :]
first_20_rows

array([[ 36.  ,   0.  ,  22.64,   6.  , 160.  ],
       [ 31.  ,   0.  ,  27.32,   5.8 , 126.  ],
       [ 52.  ,   0.  ,  23.2 ,   4.  , 100.  ],
       [ 29.  ,   0.  ,  30.57,   5.  , 160.  ],
       [ 61.  ,   0.  ,  26.88,   5.8 ,  90.  ],
       [ 36.  ,   0.  ,  21.34,   4.5 ,  90.  ],
       [ 41.  ,   0.  ,  27.32,   5.7 ,  90.  ],
       [ 54.  ,   0.  ,  18.54,   6.1 , 280.  ],
       [ 58.  ,   0.  ,  32.5 ,   6.2 , 145.  ],
       [ 71.  ,   1.  ,  44.63,   6.2 , 160.  ],
       [ 31.  ,   0.  ,  38.82,   5.  , 160.  ],
       [ 20.  ,   0.  ,  27.3 ,   5.8 ,  80.  ],
       [  2.  ,   0.  ,  18.79,   4.  , 160.  ],
       [ 43.  ,   0.  ,  27.32,   4.  ,  85.  ],
       [ 54.  ,   0.  ,  28.9 ,   6.5 , 160.  ],
       [  2.  ,   0.  ,  19.22,   4.  ,  90.  ],
       [ 60.  ,   0.  ,  30.83,   7.  , 240.  ],
       [ 30.  ,   1.  ,  27.32,   6.2 , 130.  ],
       [ 44.  ,   0.  ,  25.78,   4.8 , 155.  ],
       [ 64.  ,   1.  ,  28.98,   6.1 , 158.  ]])

In [15]:
X_test_selected = np.array([ 36, 0, 22.64, 6,160 ])
X_test_selected = X_test_selected.reshape(1, -1)
X_test_selected

array([[ 36.  ,   0.  ,  22.64,   6.  , 160.  ]])

In [16]:
y_pred=dtr.predict(first_20_rows)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])